In [1]:
import os
import sys
import time
from ast import literal_eval
from typing import Any, Dict, List, Union


sys.path.append(os.path.join(os.getcwd(), ".."))


import pandas as pd
from pprint import pprint
from dotenv import load_dotenv
from langchain_ollama import ChatOllama
from langchain_anthropic import ChatAnthropic
from langchain_huggingface import HuggingFaceEmbeddings
from ragas.dataset_schema import EvaluationDataset, EvaluationResult
from src.grag import run_text_generation_workflow, evaluate_text_generation


load_dotenv()

True

## **Preparation**

In [ ]:
OUTPUT_PATH = os.path.join("results", "llm_text_generation")
DATASET_PATH = os.path.join("data", "Dataset Testing.xlsx")

os.makedirs(OUTPUT_PATH, exist_ok=True)

df = pd.read_excel(DATASET_PATH)
dataset = []
expected_tool_call_names = []
generated_cypher_results = []

for idx, row in df.iterrows():
    if row["is_valid"]:
        dataset.append(
            {
                "user_input": str(row["user_input"]),
                "reference": str(row["reference"]),
                "retrieved_contexts": literal_eval(row["reference_contexts_1"]),
                # Atau
                # "retrieved_contexts": literal_eval(row["reference_contexts_1"]),
                # "reference_contexts": literal_eval(row["reference_contexts_1"]),
            }
        )
        # Khusus Bayu
        expected_tool_call_names.append(str(row["reference_tool_call"]))
        generated_cypher_results.append(str(row["cypher_reference"]))

evaluation_dataset = EvaluationDataset.from_list(dataset)

print(f"Jumlah soal: {len(evaluation_dataset)}")

Jumlah soal: 100


In [ ]:
CLAUDE_LLM_MODEL_NAME = "claude-3-5-haiku-20241022"
claude_llm = ChatAnthropic(
    model_name=CLAUDE_LLM_MODEL_NAME,
    max_tokens_to_sample=4096,
    temperature=0.0,
    timeout=None,
    api_key=os.environ["ANTHROPIC_API_KEY"],
)

LLAMA_LLM_MODEL_NAME = "llama3.1:8b-instruct-q4_K_M"
llama_llm = ChatOllama(
    model=LLAMA_LLM_MODEL_NAME,
    num_ctx=32768,
    num_predict=4096,
    temperature=0.0,
)

# Evaluator
EMBEDDING_MODEL_NAME = "intfloat/multilingual-e5-large"
embedding_evaluator = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)
llm_evaluator = claude_llm

## **Evaluation**

In [ ]:
def save_experiment_dataset_or_result(
    dataset: Union[EvaluationDataset, EvaluationResult],
    experiment_name: str
) -> None:
    dataset.to_pandas().to_json(
        os.path.join(OUTPUT_PATH, f"{experiment_name}.json"),
        orient="records",
    )


def run_test_case(test_case: Dict[str, Any]) -> Dict[str, Any]:
    llm_type = "api" if test_case["llm_model_name"].starts_with("claude") else "local"
    experiment_name = f"{llm_type}_{test_case["llm_model_name"]}"

    evaluation_dataset_completed = run_text_generation_workflow(
        evaluation_dataset,
        experiment_name,
        expected_tool_call_names=expected_tool_call_names,
        generated_cypher_results=generated_cypher_results,
        llm=test_case["llm_model"],
        verbose=True
    )

    # Checkpoint 1
    save_experiment_dataset_or_result(
        evaluation_dataset_completed,
        experiment_name=experiment_name
    )

    # Break 60 seconds
    time.sleep(60)

    evaluation_result = evaluate_text_generation(
        evaluation_dataset_completed,
        llm_model=llm_evaluator,
        embedding_model=embedding_evaluator,
        experiment_name=experiment_name,
    )

    # Checkpoint 2
    save_experiment_dataset_or_result(
        evaluation_result,
        experiment_name=experiment_name
    )

    return {
        "experiment_name": experiment_name,
        "args": {"llm": test_case["llm_model_name"]},
        "evaluation_result": evaluation_result,
    }

In [ ]:
test_cases = [
    {
        # Claude (API)
        "llm_model_name": CLAUDE_LLM_MODEL_NAME,
        "llm_model": claude_llm
    },
    {
        # Llama (local)
        "llm_model_name": LLAMA_LLM_MODEL_NAME,
        "llm_model": llama_llm
    }
]

## **Test Case 1**

- API: Claude

In [ ]:
test_result_1 = run_test_case(test_cases[0])

In [ ]:
pprint(test_result_1)

## **Test Case 2**

- Local: Llama

In [ ]:
test_result_2 = run_test_case(test_cases[1])

In [ ]:
pprint(test_result_2)